In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics

from warnings import simplefilter

rand_st=4
simplefilter("ignore") 

In [3]:
elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,-57,4765
1,1915,1999,1957,-84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [4]:
features=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv')
print(features.info())
print(features.shape)
features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24255 entries, 0 to 24254
Columns: 233 entries, Result to InsufficientMaterial
dtypes: float64(111), int64(122)
memory usage: 43.1 MB
None
(24255, 233)


,Result,NumMoves,MeanScore,ModeScore,FinalScore,FullMoves,MinScore,MaxScore,MedianScore,StdScore,...,EndB,EndR,EndK,WhiteСhecks,WhiteQueenCastle,BlackQueenCastle,Promotion,EnPassant,CanClaimDraw,InsufficientMaterial
count,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,...,24255.000000,24255.00000,24255.0,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000
mean,0.544877,42.252690,30.881103,11.907442,111.901711,83.967223,-794.591878,988.643950,68.913894,314.678490,...,0.631293,1.03892,1.0,2.015584,1.457308,0.910740,0.064069,0.061637,0.035415,0.000783
std,0.417736,16.235593,233.709202,445.214702,2310.851869,32.489872,2126.099646,2275.840558,64.170646,475.478471,...,0.676858,0.74847,0.0,2.874868,4.179615,3.663225,0.291749,0.245589,0.184831,0.027978
min,0.000000,10.000000,-2162.262500,-11447.000000,-12261.000000,19.000000,-12348.000000,-51.000000,0.000000,7.795716,...,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,31.000000,-60.758665,0.000000,-362.000000,61.000000,-482.000000,67.000000,30.000000,61.336328,...,0.000000,0.00000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.500000,40.000000,24.563636,0.000000,0.000000,79.000000,-75.000000,156.000000,49.000000,145.621548,...,1.000000,1.00000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,51.000000,123.987715,26.000000,546.500000,102.000000,-18.000000,700.000000,84.000000,291.317992,...,1.000000,2.00000,1.0,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,165.000000,2073.289256,11259.000000,12337.000000,329.000000,26.000000,12352.000000,784.000000,3962.575036,...,2.000000,2.00000,1.0,71.000000,35.000000,36.000000,4.000000,2.000000,1.000000,1.000000


In [5]:
target_white=elos['WhiteElo']
target_black=elos['BlackElo']
target_mean=elos['MeanElos']
target_diff=elos['DiffElos']
target_sum=elos['SumElos']

In [6]:
X_train, X_test, Y_train, Y_test =ms.train_test_split(features, target_sum, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train), len(Y_train)))
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 233)
(16978,)
(7277, 233)
(7277,)


In [7]:
def predictions_report(pred_target,predictions):
    mae_report=mae(target_sum,sum_pred)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(target_sum, sum_pred))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(target_sum,sum_pred)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return mae

In [ ]:
def GridTune(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
    
    grid = ms.GridSearchCV(classifiers,
                           params,  # настройка параметров через кросс-валидацию
                           refit=True,    # переобучение по лучшим найденным параметрам   
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))
    
    grid_best_model = grid.fit(X_train,Y_train)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [ ]:
rf_params =[{'n_estimators':[50, 100, 200,500,1000],'criterion':['mae', 'mse'],'max_depth':[None, 5, 10,15],
             'min_samples_leaf':[1,2,4,7,10],'max_features':[0.25,0.5,1, 2, 3,4],'bootstrap':[True],'oob_score':[True,False],
             'min_samples_split':[0.5,1,2,5,10,15,20], 'max_leaf_nodes':[None, 5,10,100,500,1000]},
           {'n_estimators':[50, 100, 200,500,1000],'criterion':['mae', 'mse'],'max_depth':[None, 5, 10,15],
             'min_samples_leaf':[1,2, 4,7,10],'max_features':[0.25,0.5, 1, 2, 3,4],'bootstrap':[False],
               'min_samples_split':[0.5,1,2,5,10,15,20], 'max_leaf_nodes':[None, 5,10,100,500,1000]}]

rf=RandomForestRegressor(random_state=rand_st,n_jobs=-1)
rf_model=GridTune(rf_params,rf,10)
rf_predictions=rf_model.predict(X_test)
dict_model['RandomForest']=predictions_report(Y_test,rf_predictions)  